
## 臺灣車流追蹤系統

## 車輛計數-OpenCV
        使用 YOLOv8 模型和物件追蹤框架 (abewley/sort) 從影片中偵測和追蹤汽車。
        檢測和追蹤是在感興趣的區域進行的。
        YOLOv8 是最新、最先進的 YOLO 模型，可用於物件偵測、影像分類和實例分割任務。
        SORT 是基於基本資料關聯和狀態估計技術的視覺多物件追蹤框架的準系統實作。

## 參考連結
    https://github.com/redeagle17/Car-Counting
    使用 YOLO 模型進行物體檢測，使用 Sort 算法進行目標追蹤，同時計算穿越指定計數線的車輛數量。

## classes 類別有哪些?

    
    classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "potted plant", "bed",
              "dining table", "toilet", "tv monitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"]

### 使用套件

In [25]:
from ultralytics import YOLO
from sort import *
import cv2
import cvzone
import math
import numpy as np

In [26]:
#Open Video
cap = cv2.VideoCapture("./KHheighway_video/721836327.118740.mp4")
#寫入一個 VideoWriter 對象，指定儲存的檔案名稱、影片編碼方式、影片幀率和影片尺寸
out = cv2.VideoWriter('output_video.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 10, (int(cap.get(3)),int(cap.get(4))))


#定義 YOLO 模型預測結果中可能的類別名稱列表 
classNames = ["car", "motorbike", "bus", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "potted plant", "bed",
              "dining table", "toilet", "tv monitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"]

#Load .pt權重檔案
model = YOLO("yolov8n.pt")


## 在這裡，mask 是遮罩影像，img 是原始影像。 
    位元與運算的規則是，如果兩個對應位元都是 1，則結果中的該位元也是 1，否則為 0。 在這裡，遮罩影像中非零的像素將保留，而原始影像中對應位置的像素將被保留或忽略，這取決於遮罩影像中對應位置的值。

    這樣，imgRegion 將包含原始影像中與遮罩影像中非零像素對應的區域，其他區域將被遮罩掉。

    這種操作通常用於在影像上提取或突出顯示特定區域，或將影像分割為感興趣的區域。
    在這裡，是為了在檢測或計數之前，只關注影像中的特定區域，以提高處理效率或減少誤檢測。

In [27]:
#從檔案讀取一個遮罩圖像 (mask.png)，用於指定計數區域。
#mask = cv2.imread("mask.png")
mask = cv2.imread("mask.png")


In [28]:
# 用來累計各class
car_count = 0
bus_count = 0
truck_count = 0

In [29]:
#初始化 Sort 目標追蹤器，設定最大追蹤幀數 (max_age) 和最小追蹤點數 (min_hits)。
tracker = Sort(max_age=70, min_hits=3)

In [30]:
#定義計數線的兩個端點的座標，定義計數線的座標，[x1, y1, x2, y2]，此處表示一條水平的計數線。
lines_coordinates = [300, 420, 630, 420]  # lines_coordinates[0] is one point and lines_coordinates[2] is another point

In [31]:
# lines_coordinates[1] is height from origin and lines_coordinates[2] is height from origin
totalCount = 0
visited_id_list = [] #初始化已訪問過的目標 ID 列表

In [32]:
while True:
    success, img = cap.read() #讀取影片的一個影格
    imgRegion = cv2.bitwise_and(mask, img)  # 將遮罩和影格進行位元運算，獲取檢測區域
    results = model(imgRegion, stream=True)  #使用 YOLO 模型檢測區域中的物體。
    
    #讀取圖形影像，並將其覆蓋到原始影像上。(左上繳裝飾)
    imgGraphics = cv2.imread("graphics.png", cv2.IMREAD_UNCHANGED)
    img = cvzone.overlayPNG(img, imgGraphics, (0, 0))
    
    #初始化一個空的數組，用於存儲檢測結果
    detections = np.empty((0, 5))

    for r in results:
        boxes = r.boxes
        for box in boxes:
            # To draw bounding box 獲取檢測框的坐標
            x1, y1, x2, y2 = box.xyxy[0]  # Gives coordinates to draw bounding box 
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            #cv2.rectangle(img, (x1, y1), (x2, y2), (255, 255, 0), 3)  顯示檢測框
            
            #計算檢測框的寬度和高度
            w, h = x2 - x1, y2 - y1
            bbox = (x1, y1, w, h)

            # Annotation is required for the detected object 計算檢測框的可信度，並四捨五入至兩位小數。
            confidence_level = math.ceil((box.conf[0] * 100)) / 100  # rounding of to 2 decimal places

            # To get class name 獲取檢測框的類別名稱。
            cls = int(box.cls[0])  # box.cls[0] will give us the class id(float so we need to typecast it) and with
            
            
            # the help of the id will get the class name from classNames list
            currentClass = classNames[cls]

            #如果檢測到的物體是車輛類別且信度大於 0.3，則進行以下操作。
            if currentClass == "car" or currentClass == "motorbike" or currentClass == "truck" or \
                    currentClass == "bus" and confidence_level > 0.3:
                cvzone.putTextRect(img, f'{currentClass} {confidence_level}', (max(0, x1), max(35, y1)), scale=0.6,
                                    thickness=2, offset=3)
                cvzone.cornerRect(img, bbox, l=9)
                # cornerRect is the  built-in function in cvzone to draw bounding box

                #將檢測結果添加到 detections 數組中
                currentArray = np.array([x1, y1, x2, y2, confidence_level])  # Check update function in sort.py
                detections = np.vstack((detections, currentArray))

    #將檢測結果添加到 detections 數組中            
    resultTracker = tracker.update(detections)

    #在影像上各畫一條紅、綠、藍色的計數線BGR
    cv2.line(img, (lines_coordinates[0], lines_coordinates[1]), (lines_coordinates[2], lines_coordinates[3]),
             (0, 0, 255), thickness=2)
    cv2.line(img, (lines_coordinates[0], lines_coordinates[1]-30), (lines_coordinates[2], lines_coordinates[3]-30),
             (0, 255, 0), thickness=2)
    cv2.line(img, (lines_coordinates[0], lines_coordinates[1]+30), (lines_coordinates[2], lines_coordinates[3]+30),
             (255, 0, 0), thickness=2)

    #處理目標追蹤結果
    for result in resultTracker:
        #獲取目標追蹤結果的座標、寬度和高度
        x1, y1, x2, y2, id = result
        x1, y1, x2, y2, id = int(x1), int(y1), int(x2), int(y2), int(id)
        w, h = x2 - x1, y2 - y1
        # print(result)
        #在影像上標記目標的 ID
        cvzone.putTextRect(img, f'{id}', (max(0, x1), max(35, y1)), scale=2.3,
                           thickness=2, offset=3)
        #在目標的檢測框上畫一個彩色的矩形。
        cvzone.cornerRect(img, (x1, y1, w, h), l=9, t=3, rt=2, colorR=(255, 0, 255))

        #在目標的中心點處畫一個滿填充的圓形
        # Now, we want the center point of the detected object and if it touches the line then increase the count
        cx, cy = x1 + w // 2, y1 + h // 2
        cv2.circle(img, (cx, cy), 4, (255, 0, 255), cv2.FILLED)

        #如果目標的中心點碰觸到計數線，則進行以下操作。
        #可以設置計數範圍
        if lines_coordinates[0] < cx < lines_coordinates[2] and lines_coordinates[1] - 30 < cy < lines_coordinates[
            1] + 30:
            
            #如果目標的 ID 還未被訪問，則增加計數，標記已訪問的 ID，並在影像上畫一條綠色的計數線。
            if id not in visited_id_list:  # To count the number only once
                
                #不知道為什麼都是偵測到currentClass == "bus"?
                #if currentClass == "car":
                #    car_count += 1
                #elif currentClass == "bus":
                #    bus_count += 1
                #elif currentClass == "truck":
                #    truck_count += 1
                
                totalCount = totalCount + 1
                visited_id_list.append(id)
                cv2.line(img, (lines_coordinates[0], lines_coordinates[1]),
                         (lines_coordinates[2], lines_coordinates[3]),
                         (0, 255, 0), thickness=2)
   
    #在影像上顯示計數值
    cv2.putText(img, str(totalCount), (255, 100), cv2.FONT_HERSHEY_PLAIN, 5, (0, 0, 0), 8)
    #cv2.putText(img, f"Car Count: {car_count}", (255, 150), cv2.FONT_HERSHEY_PLAIN, 5, (0, 0, 0), 8)
    #cv2.putText(img, f"Bus Count: {bus_count}", (255, 200), cv2.FONT_HERSHEY_PLAIN, 5, (0, 0, 0), 8)
    #cv2.putText(img, f"Truck Count: {truck_count}", (255, 250), cv2.FONT_HERSHEY_PLAIN, 5, (0, 0, 0), 8)
    #cvzone.putTextRect(img, f'Count {totalCount}', (40, 40), colorT=(255, 255, 255), colorR=(0, 0, 0))

    # 寫入影格到 VideoWriter 對象
    out.write(img)

    cv2.imshow("Image", img)
    # cv2.imshow("Region", imgRegion)
    #cv2.waitKey(1)

    #檢查是否按下 'q' 鍵，如果是則釋放影片捕捉器並跳出迴圈
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break    
cv2.destroyAllWindows()


0: 384x640 1 truck, 116.8ms
Speed: 2.5ms preprocess, 116.8ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 truck, 150.0ms
Speed: 4.0ms preprocess, 150.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 103.1ms
Speed: 3.0ms preprocess, 103.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 truck, 106.5ms
Speed: 2.0ms preprocess, 106.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 109.5ms
Speed: 2.0ms preprocess, 109.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 108.1ms
Speed: 2.0ms preprocess, 108.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 truck, 104.8ms
Speed: 2.0ms preprocess, 104.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 truck, 107.7ms
Speed: 3.0ms preprocess, 107.7ms inference, 2.0ms postpro

# 一鍵完工
    #參考連結:https://github.com/redeagle17/Car-Counting
    #使用 YOLO 模型進行物體檢測，使用 Sort 算法進行目標追蹤，同時計算穿越指定計數線的車輛數量。
    from ultralytics import YOLO
    from sort import *
    import cv2
    import cvzone
    import math
    import numpy as np

    #Open Video
    cap = cv2.VideoCapture("./KHheighway_video/721836327.118740.mp4")
    #開啟一個 VideoWriter 對象，指定儲存的檔案名稱、影片編碼方式、影片幀率和影片尺寸
    out = cv2.VideoWriter('output_video.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 10, (int(cap.get(3)),int(cap.get(4))))


    #定義 YOLO 模型預測結果中可能的類別名稱列表 
    classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
                "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
                "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
                "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
                "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
                "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
                "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "potted plant", "bed",
                "dining table", "toilet", "tv monitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
                "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
                "teddy bear", "hair drier", "toothbrush"]

    #Load .pt權重檔案
    model = YOLO("yolov8n.pt")

    #從檔案讀取一個遮罩圖像 (mask.png)，用於指定計數區域。
    mask = cv2.imread("mask.png")

    #初始化 Sort 目標追蹤器，設定最大追蹤幀數 (max_age) 和最小追蹤點數 (min_hits)。
    tracker = Sort(max_age=70, min_hits=3)

    #定義計數線的兩個端點的座標，定義計數線的座標，[x1, y1, x2, y2]，此處表示一條水平的計數線。
    lines_coordinates = [300, 420, 630, 420]  # lines_coordinates[0] is one point and lines_coordinates[2] is another point

    #lines_coordinates[1] is height from origin and lines_coordinates[2] is height from origin
    totalCount = 0
    visited_id_list = [] #初始化已訪問過的目標 ID 列表

    while True:
        success, img = cap.read() #讀取影片的一個影格
        imgRegion = cv2.bitwise_and(mask, img)  # 將遮罩和影格進行位元運算，獲取檢測區域
        results = model(imgRegion, stream=True)  #使用 YOLO 模型檢測檢測區域中的物體。
        
        #讀取圖形影像，並將其覆蓋到原始影像上。(左上繳裝飾)
        imgGraphics = cv2.imread("graphics.png", cv2.IMREAD_UNCHANGED)
        img = cvzone.overlayPNG(img, imgGraphics, (0, 0))
        
        #初始化一個空的數組，用於存儲檢測結果
        detections = np.empty((0, 5))

        for r in results:
            boxes = r.boxes
            for box in boxes:
                # To draw bounding box 獲取檢測框的坐標
                x1, y1, x2, y2 = box.xyxy[0]  # Gives coordinates to draw bounding box 
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                # cv2.rectangle(img, (x1, y1), (x2, y2), (255, 255, 0), 3)
                
                #計算檢測框的寬度和高度
                w, h = x2 - x1, y2 - y1
                bbox = (x1, y1, w, h)

                # Annotation is required for the detected object 計算檢測框的置信度，並四捨五入至兩位小數。
                confidence_level = math.ceil((box.conf[0] * 100)) / 100  # rounding of to 2 decimal places

                # To get class name 獲取檢測框的類別名稱。
                cls = int(box.cls[0])  # box.cls[0] will give us the class id(float so we need to typecast it) and with
                
                
                # the help of the id will get the class name from classNames list
                currentClass = classNames[cls]

                #如果檢測到的物體是車輛類別且信度大於 0.3，則進行以下操作。
                if currentClass == "car" or currentClass == "motorbike" or currentClass == "truck" or \
                        currentClass == "bus" and confidence_level > 0.3:
                    cvzone.putTextRect(img, f'{currentClass} {confidence_level}', (max(0, x1), max(35, y1)), scale=0.6,
                                        thickness=2, offset=3)
                    cvzone.cornerRect(img, bbox, l=9)
                    # cornerRect is the  built-in function in cvzone to draw bounding box

                    #將檢測結果添加到 detections 數組中
                    currentArray = np.array([x1, y1, x2, y2, confidence_level])  # Check update function in sort.py
                    detections = np.vstack((detections, currentArray))

        #將檢測結果添加到 detections 數組中            
        resultTracker = tracker.update(detections)

        #在影像上畫一條紅色的計數線BGR
        cv2.line(img, (lines_coordinates[0], lines_coordinates[1]), (lines_coordinates[2], lines_coordinates[3]),
                (0, 0, 255), thickness=2)

        #處理目標追蹤結果
        for result in resultTracker:
            #獲取目標追蹤結果的座標、寬度和高度
            x1, y1, x2, y2, id = result
            x1, y1, x2, y2, id = int(x1), int(y1), int(x2), int(y2), int(id)
            w, h = x2 - x1, y2 - y1
            # print(result)
            #在影像上標記目標的 ID
            cvzone.putTextRect(img, f'{id}', (max(0, x1), max(35, y1)), scale=2.3,
                            thickness=2, offset=3)
            #在目標的檢測框上畫一個彩色的矩形。
            cvzone.cornerRect(img, (x1, y1, w, h), l=9, t=3, rt=2, colorR=(255, 0, 255))

            #在目標的中心點處畫一個滿填充的圓形
            # Now, we want the center point of the detected object and if it touches the line then increase the count
            cx, cy = x1 + w // 2, y1 + h // 2
            cv2.circle(img, (cx, cy), 4, (255, 0, 255), cv2.FILLED)

            #如果目標的中心點碰觸到計數線，則進行以下操作。
            if lines_coordinates[0] < cx < lines_coordinates[2] and lines_coordinates[1] - 10 < cy < lines_coordinates[
                1] + 10:
                #如果目標的 ID 還未被訪問，則增加計數，標記已訪問的 ID，並在影像上畫一條綠色的計數線。
                if id not in visited_id_list:  # To count the number only once
                    totalCount = totalCount + 1
                    visited_id_list.append(id)
                    cv2.line(img, (lines_coordinates[0], lines_coordinates[1]),
                            (lines_coordinates[2], lines_coordinates[3]),
                            (0, 255, 0), thickness=2)
    
        #在影像上顯示計數值
        cv2.putText(img, str(totalCount), (255, 100), cv2.FONT_HERSHEY_PLAIN, 5, (0, 0, 0), 8)
        #cvzone.putTextRect(img, f'Count {totalCount}', (40, 40), colorT=(255, 255, 255), colorR=(0, 0, 0))

        # 寫入影格到 VideoWriter 對象
        out.write(img)

        cv2.imshow("Image", img)
        # cv2.imshow("Region", imgRegion)
        #cv2.waitKey(1)

        #檢查是否按下 'q' 鍵，如果是則釋放影片捕捉器並跳出迴圈
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            break    
    cv2.destroyAllWindows()